# Mongo tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Mongo commands](https://docs.mongodb.com/manual/reference/)
* [Mongo python client](http://api.mongodb.com/python/current/api/pymongo/mongo_client.html#pymongo.mongo_client.MongoClient)

### Import libraries

In [1]:
import datetime
from pprint import pprint

import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27018)

In [3]:
# let's work in a test_database
db = client.test_database
posts = db.posts

In [4]:
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('6580192c58ea6ccdc748ee5d')

In [5]:
db.list_collection_names()

['posts']

In [6]:
pprint(posts.find_one())

{'_id': ObjectId('6580192c58ea6ccdc748ee5d'),
 'author': 'Mike',
 'date': datetime.datetime(2023, 12, 18, 10, 4, 28, 184000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


You can launch a terminal aside, connect to your server with a mongo client and check that the value is present :

```bash
vagrant@nosql:~$ mongo
> show databases;
admin          0.000GB
config         0.000GB
local          0.000GB
test_database  0.000GB
> use test_database;
switched to db test_database
> db.posts.find()
{ 
    "_id" : ObjectId("..."), 
    "author" : "Mike", 
    "text" : "My first blog post!", 
    "tags" : [ "mongodb", "python", "pymongo"], 
    "date" : ISODate("2019-02-10T11:33:47.883Z") 
}
```

## I. Quick start

### First steps

**Q** : Create a document `{msg: 'hello'}` in the `test` collection with `insert_one()`. Fetch it back to display it. What is the `_id` for ?

NB : if the collection doesn't exist yet, MongoDB automatically creates it.

In [7]:
tests = db.test
doc = {'msg': 'hello'}
doc_id = tests.insert_one(doc).inserted_id
doc_id

ObjectId('6580194358ea6ccdc748ee5e')

**Q**: Display the number of documents inside the `test` collection

In [8]:
tests.count_documents({})

1

### Interacting with a database

We have 2 `.json` files we want to interact with inside the `data` folder. Let's first dump them into a `MovieLens` database, inside `users` and `movies` collections.

For this section, you will need to read a bit on [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors). Most methods on collections you will use have `filter` as a first parameter, on which you must pass a dictionary of query parameters.

**Q** : In the `MovieLens` database, load `data/movielens_movies.json` into `movies` and `data/movielens_users.json` into `users`. 

Use the dedicated shell command for this : `mongoimport --db <some_db> --collection <some_collection> --file <some_file>` 

In [ ]:
!docker cp ../data/movielens_users.json some-mongo:/data/movielens_users.json
!docker exec some-mongo mongoimport --db movie_lens --collection users --file /data/movielens_users.json

In [ ]:
!docker cp ../data/movielens_movies.json some-mongo:/data/movielens_movies.json
!docker exec some-mongo mongoimport --db movie_lens --collection movies --file /data/movielens_movies.json

**Q** : how many users are in the `MovieLens` database ?

In [18]:
movie_lens = client.movie_lens
users = movie_lens.users
movies = movie_lens.movies
users.count_documents({})

6040

**Q** : Display all comedies (the `genres` property equals `Comedy`). 

NB : You will need to find how to go through a `command_cursor`, then use the `pprint` function for a better display of those documents.

In [21]:
users.count_documents({})

6040

In [25]:
comedies = movies.find({'genres': 'Comedy'})
for movie in comedies:
    pprint(movie)

{'_id': 5, 'genres': 'Comedy', 'title': 'Father of the Bride Part II (1995)'}
{'_id': 19,
 'genres': 'Comedy',
 'title': 'Ace Ventura: When Nature Calls (1995)'}
{'_id': 38, 'genres': 'Comedy', 'title': 'It Takes Two (1995)'}
{'_id': 52, 'genres': 'Comedy', 'title': 'Mighty Aphrodite (1995)'}
{'_id': 65, 'genres': 'Comedy', 'title': 'Bio-Dome (1996)'}
{'_id': 69, 'genres': 'Comedy', 'title': 'Friday (1995)'}
{'_id': 88, 'genres': 'Comedy', 'title': 'Black Sheep (1996)'}
{'_id': 96, 'genres': 'Comedy', 'title': 'In the Bleak Midwinter (1995)'}
{'_id': 101, 'genres': 'Comedy', 'title': 'Bottle Rocket (1996)'}
{'_id': 63,
 'genres': 'Comedy',
 'title': "Don't Be a Menace to South Central While Drinking Your Juice in the "
          'Hood (1996)'}
{'_id': 109, 'genres': 'Comedy', 'title': 'Headless Body in Topless Bar (1995)'}
{'_id': 115, 'genres': 'Comedy', 'title': 'Happiness Is in the Field (1995)'}
{'_id': 102, 'genres': 'Comedy', 'title': 'Mr. Wrong (1996)'}
{'_id': 119, 'genres': 'C

**Q** : Fetch and display the `name` and `occupation` for Clifford Johnathan. The second paramater for `find()` ([doc here](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)) is called the `projection` and is used to limit which data to fetch from the query.

In [31]:
clifford = users.find_one({'name': 'Clifford Johnathan'}, {'_id': False, 'name': True, 'occupation': True})
pprint(clifford)

{'name': 'Clifford Johnathan', 'occupation': 'technician/engineer'}


**Q**: How many minors (by `age`) have rated movies ?

In [33]:
users.count_documents({'age': {'$lt': 18}})

222

**Q**: Display science fiction movies ('Sci-Fi') and suspense movies ('Thriller'). This time you need to use a regex to parse genres and look for those values.

In [44]:
fiction_and_suspense = movies.find({'genres': {'$regex': 'Sci-Fi|Thriller'}})
for movie in fiction_and_suspense:
    pprint(movie)

{'_id': 6, 'genres': 'Action|Crime|Thriller', 'title': 'Heat (1995)'}
{'_id': 10, 'genres': 'Action|Adventure|Thriller', 'title': 'GoldenEye (1995)'}
{'_id': 16, 'genres': 'Drama|Thriller', 'title': 'Casino (1995)'}
{'_id': 29,
 'genres': 'Adventure|Sci-Fi',
 'title': 'City of Lost Children, The (1995)'}
{'_id': 32, 'genres': 'Drama|Sci-Fi', 'title': 'Twelve Monkeys (1995)'}
{'_id': 22, 'genres': 'Crime|Drama|Thriller', 'title': 'Copycat (1995)'}
{'_id': 24, 'genres': 'Drama|Sci-Fi', 'title': 'Powder (1995)'}
{'_id': 18, 'genres': 'Thriller', 'title': 'Four Rooms (1995)'}
{'_id': 23, 'genres': 'Thriller', 'title': 'Assassins (1995)'}
{'_id': 47, 'genres': 'Crime|Thriller', 'title': 'Seven (Se7en) (1995)'}
{'_id': 50, 'genres': 'Crime|Thriller', 'title': 'Usual Suspects, The (1995)'}
{'_id': 51, 'genres': 'Action|Drama|Thriller', 'title': 'Guardian Angel (1994)'}
{'_id': 76, 'genres': 'Sci-Fi|Thriller', 'title': 'Screamers (1995)'}
{'_id': 70,
 'genres': 'Action|Comedy|Crime|Horror|Thri

**Q**: If we want more advanced textual search, we need a particular index. Use the `create_index()` method to index as [TEXT](https://docs.mongodb.com/manual/core/index-text/) the `genres` field of the `movies` collection.

In [52]:
movies.create_index([('genres', 1)])

'genres_1'

**Q**: Restart the search for science fiction and thriller movies with the operator `$text`

**Q**: Display the first 30 movies (`limit`) in alphabetical order (`sort`) by title

**Q**: How many users have seen the movie "Star Wars: Episode V - The Empire Strikes Back (1980)" (`_id 1196`) ? The `movies` argument is an array so we should try the [elemMatch](https://docs.mongodb.com/manual/reference/operator/projection/elemMatch/) operator here.

**Q**: And how many gave it a rating of 1 or 2 ?

### Updating data

**Q**: Insert a new user with the properties `name`, `gender` ('M' or'F'), `occupation` and `age`, using the `insert_one()` command. Display it with `find_one()`.

**Q**: Add an appreciation on a viewed movie with `update_one()`, add the movies property containing a table with a document (`movieid`, `rating`, `timestamp` with the value `datetime.datetime.utcnow()`).

You will need to read the documentation on [update operators](https://docs.mongodb.org/manual/reference/operator/update/).

**Q**: Find the number of users who have declared a `programmer` occupation. Modify them so that they are `developer`. Verify your update.

## II. Modelling a blog

We will now model a blog using Mongo. 

First, switch to a new `Blog` database. Each blog post will have the following arguments:

* The author (author field, string type)
* The date (date field, string type in YYYY-MM-DD format)
* The content (field content)
* Tags (field tags, a string array)
* A list of comments (field comments) containing:
 * The author (author field, string type)
 * The date (date field, string type in YYYY-MM-DD format)
 * The content (field content)


**Q**: Create a first post by `rick`, on January 15th, with the tags `mongodb` and `nosql`.

**Q**: Create a second post by `kate`, on January 21, with the tag `nosql` and a comment from `rick` on the same day.

**Q**: Display the author of the last post with the tag `nosql`

**Q**: Add a comment by `jack` on January 25, to `kate`'s post

**Q**: Display all comments by `kate`

## Postquisites

In [ ]:
!mongo test_database --eval 'db.dropDatabase()'

In [ ]:
!mongo MovieLens --eval 'db.dropDatabase()'

In [ ]:
!mongo Blog --eval 'db.dropDatabase()'